# Chapter 7: The Transformer Block

## Bringing It All Together

In the previous notebooks, we built each component individually:
- **Causal Attention** - How tokens look at previous tokens
- **Multi-Head Attention** - Multiple parallel attention patterns  
- **Layer Normalization** - Stabilizing activations
- **GELU Activation** - Smooth non-linearity
- **Feed-Forward Network** - Processing each position

Now it's time to **assemble these building blocks** into a complete **Transformer Block** - the fundamental unit that gets stacked to create GPT!

```
┌─────────────────────────────────────────────────────────────┐
│                    TRANSFORMER BLOCK                        │
│                                                             │
│    Input ──┬──► LayerNorm ──► Multi-Head ──► Dropout ──┐   │
│            │                  Attention                 │   │
│            │                                            │   │
│            └────────────────── + ◄──────────────────────┘   │
│                                │                            │
│                                ▼                            │
│            ┌──► LayerNorm ──► FeedForward ──► Dropout ──┐   │
│            │                                             │   │
│            │                                             │   │
│            └────────────────── + ◄───────────────────────┘   │
│                                │                            │
│                                ▼                            │
│                             Output                          │
└─────────────────────────────────────────────────────────────┘
```

## Key Insight: The "Pre-LN" Architecture

GPT-2 uses **Pre-Layer Normalization** (Pre-LN), where LayerNorm comes *before* each sub-layer:

| Architecture | Order | Advantage |
|-------------|-------|-----------|
| **Post-LN** (Original) | Attention → Add → Norm | Harder to train deep networks |
| **Pre-LN** (GPT-2) | Norm → Attention → Add | More stable training |

---

## Setup

In [1]:
!pip install torch tiktoken transformers

---

## Building Block 1: Multi-Head Attention (Recap)

This is the same Multi-Head Attention we built in Chapter 5. Let's recall what it does:

### What Multi-Head Attention Computes

Each **head** learns to focus on different aspects of the input:
- Head 1 might focus on **syntax** (subject-verb relationships)
- Head 2 might focus on **semantics** (word meaning connections)
- Head 3 might focus on **position** (nearby words)

### The Efficient Implementation

Instead of running `num_heads` separate attention modules, we:

1. **Project once** with large matrices (d_in → d_out)
2. **Reshape** to split into heads: `(batch, seq, d_out)` → `(batch, seq, num_heads, head_dim)`
3. **Transpose** to get heads dimension first: `(batch, num_heads, seq, head_dim)`
4. **Compute attention** for all heads in parallel
5. **Reshape back** and project output

```
Input: (batch=2, tokens=4, d_in=768)
                    │
        ┌───────────┼───────────┐
        ▼           ▼           ▼
     W_query     W_key      W_value
        │           │           │
        ▼           ▼           ▼
   (2,4,768)   (2,4,768)   (2,4,768)
        │           │           │
        └───────────┼───────────┘
                    │ reshape to 12 heads
                    ▼
            (2, 12, 4, 64)  ◄── 768 ÷ 12 = 64 per head
                    │
              Attention
                    │
                    ▼
            (2, 12, 4, 64)
                    │ reshape back
                    ▼
              (2, 4, 768)
                    │
               out_proj
                    │
                    ▼
         Output: (2, 4, 768)
```

### Key Features in This Implementation

| Feature | Purpose |
|---------|---------|
| `qkv_bias=False` | GPT-2 doesn't use bias in Q, K, V projections |
| Causal mask | Prevents attending to future tokens |
| Dropout | Regularization on attention weights |
| `out_proj` | Final linear projection to mix head outputs |

In [4]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)  
        queries = queries.view(                                             
            b, num_tokens, self.num_heads, self.head_dim                    
        )                                                                   

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        context_vec = self.out_proj(context_vec)
        return context_vec

---

## Building Block 2: Layer Normalization (Recap)

Layer Normalization is crucial for **stable training** of deep transformers.

### Why We Need Normalization

As data flows through many layers, values can:
- **Explode** → gradients become NaN
- **Vanish** → gradients become 0, no learning

LayerNorm keeps values in a stable range after each sub-layer.

### How It Works

For each token independently, normalize across the embedding dimension:

```
                    Token embedding (768 values)
                    ┌─────────────────────────────┐
                    │ 2.1  -0.3  1.5  ...  0.8   │
                    └─────────────────────────────┘
                              │
                    ┌─────────▼─────────┐
                    │ Compute mean (μ)   │ → single value
                    │ Compute var (σ²)   │ → single value  
                    └─────────┬─────────┘
                              │
                    ┌─────────▼─────────┐
                    │  (x - μ)          │
                    │  ─────────        │  Normalize
                    │  √(σ² + ε)        │
                    └─────────┬─────────┘
                              │
                    ┌─────────▼─────────┐
                    │ scale * x + shift │  Learnable params
                    └─────────┬─────────┘
                              │
                    ┌─────────▼─────────────────────┐
                    │ 0.5  -0.1  0.3  ...  0.2     │  Normalized
                    └───────────────────────────────┘
```

### The Learnable Parameters

| Parameter | Shape | Initial Value | Purpose |
|-----------|-------|---------------|---------|
| `scale` | (768,) | 1.0 | Allow network to "undo" normalization if needed |
| `shift` | (768,) | 0.0 | Allow shifting the mean |

These let the network learn the optimal distribution for each feature!

In [5]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

---

## Building Block 3: GELU Activation (Recap)

**GELU** (Gaussian Error Linear Unit) is the activation function used in GPT-2, BERT, and most modern transformers.

### Why Not ReLU?

| Activation | Formula | Problem |
|------------|---------|---------|
| **ReLU** | max(0, x) | "Dead neurons" - once output is 0, gradient is 0 forever |
| **GELU** | x · Φ(x) | Smooth, no dead neurons, probabilistic interpretation |


### The GELU Formula

GELU can be approximated as:

```
GELU(x) ≈ 0.5 * x * (1 + tanh(√(2/π) * (x + 0.044715 * x³)))
```

This smooth curve:
- Acts like identity for large positive x
- Acts like 0 for large negative x  
- Allows **small gradients** for small negative values (unlike ReLU!)

In [6]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

---

## Building Block 4: Feed-Forward Network (Recap)

The Feed-Forward Network (FFN) is a simple but powerful component applied to **each token position independently**.

### The "Expand then Contract" Pattern

```
    Token embedding                    Token embedding
       (768)                              (768)
         │                                  ▲
         ▼                                  │
    ┌─────────┐                        ┌─────────┐
    │ Linear  │  768 → 3072            │ Linear  │  3072 → 768
    │  (4x)   │                        │  (÷4)   │
    └────┬────┘                        └────┬────┘
         │                                  │
         ▼                                  │
    ┌─────────┐                             │
    │  GELU   │  Non-linearity              │
    └────┬────┘                             │
         │                                  │
         └──────────────────────────────────┘
```

### Why Expand 4x?

The expansion factor of 4 is a **design choice** that works well empirically:

| Layer | Dimensions | Parameters |
|-------|-----------|------------|
| Linear 1 | 768 → 3072 | 768 × 3072 = 2.4M |
| Linear 2 | 3072 → 768 | 3072 × 768 = 2.4M |
| **Total** | | **4.8M per block** |

The larger intermediate dimension allows the network to:
- Learn more complex transformations
- Store more "knowledge" in the weights
- Act as a **key-value memory** (recent research insight!)

In [7]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

---

## 🎯 The Main Event: TransformerBlock

Now we combine all four building blocks into a single **TransformerBlock**!

### The Architecture (Pre-LN Style)

```
                         Input x
                            │
           ┌────────────────┼────────────────┐
           │                │                │
           │         ┌──────▼──────┐         │
           │         │  LayerNorm  │         │
           │         └──────┬──────┘         │
           │                │                │
           │         ┌──────▼──────┐         │
           │         │   Multi-    │         │
           │         │    Head     │         │
           │         │  Attention  │         │
           │         └──────┬──────┘         │
           │                │                │
           │         ┌──────▼──────┐         │
           │         │   Dropout   │         │
           │         └──────┬──────┘         │
           │                │                │
           └───────────► (+) ◄───────────────┘
                            │ ◄── Residual Connection 1
           ┌────────────────┼────────────────┐
           │                │                │
           │         ┌──────▼──────┐         │
           │         │  LayerNorm  │         │
           │         └──────┬──────┘         │
           │                │                │
           │         ┌──────▼──────┐         │
           │         │ FeedForward │         │
           │         └──────┬──────┘         │
           │                │                │
           │         ┌──────▼──────┐         │
           │         │   Dropout   │         │
           │         └──────┬──────┘         │
           │                │                │
           └───────────► (+) ◄───────────────┘
                            │ ◄── Residual Connection 2
                            ▼
                         Output
```

### Understanding the Code

Let's break down the `forward` method step by step:

```python
def forward(self, x):
    # ═══════════════════════════════════════════════════
    # PART 1: Attention Sub-Block with Residual
    # ═══════════════════════════════════════════════════
    shortcut = x              # Save input for residual
    x = self.norm1(x)         # Pre-LayerNorm (normalize first!)
    x = self.att(x)           # Multi-Head Attention
    x = self.drop_shortcut(x) # Dropout for regularization
    x = x + shortcut          # Add residual (skip connection)
    
    # ═══════════════════════════════════════════════════
    # PART 2: FeedForward Sub-Block with Residual  
    # ═══════════════════════════════════════════════════
    shortcut = x              # Save for second residual
    x = self.norm2(x)         # Pre-LayerNorm
    x = self.ff(x)            # FeedForward network
    x = self.drop_shortcut(x) # Dropout
    x = x + shortcut          # Add residual
    
    return x
```

### Why Residual Connections Are Critical

Without residual connections, training 12+ layer transformers is nearly impossible:

| Depth | Without Residual | With Residual |
|-------|-----------------|---------------|
| 6 layers | Trainable | Trainable |
| 12 layers | Vanishing gradients | Trainable ✓ |
| 24 layers | Won't converge | Trainable ✓ |
| 96 layers | Impossible | Trainable ✓ |

The residual connection creates a "gradient highway" that allows gradients to flow directly backward through the network!

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x

---

## GPT-2 Configuration (124M Parameters)

Before we can instantiate our TransformerBlock, we need to define the configuration. This is the same config we'll use for the full GPT model:

| Parameter | Value | Description |
|-----------|-------|-------------|
| `vocab_size` | 50,257 | Number of unique tokens (BPE vocabulary) |
| `context_length` | 1,024 | Maximum sequence length |
| `emb_dim` | 768 | Embedding dimension (d_model) |
| `n_heads` | 12 | Number of attention heads |
| `n_layers` | 12 | Number of transformer blocks |
| `drop_rate` | 0.1 | Dropout probability (10%) |
| `qkv_bias` | False | No bias in Q, K, V projections |

### Parameter Count for One TransformerBlock

Let's calculate how many parameters are in a single block:

```
Multi-Head Attention:
  - W_query:  768 × 768 = 590,000
  - W_key:    768 × 768 = 590,000
  - W_value:  768 × 768 = 590,000
  - out_proj: 768 × 768 = 590,000
  Subtotal: ~2.4M

FeedForward:
  - Linear1: 768 × 3072 = 2,360,000
  - Linear2: 3072 × 768 = 2,360,000
  Subtotal: ~4.7M

LayerNorm (×2):
  - scale + shift: 768 × 2 × 2 = 3,000

Total per block: ~7.1M parameters
Total for 12 blocks: ~85M (+ embeddings = 124M)
```

In [10]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

---

## Testing Our TransformerBlock

Let's verify that our TransformerBlock works correctly by passing through a random tensor:

- **Input**: Random tensor of shape `(batch=2, tokens=4, emb_dim=768)`
- **Expected output**: Same shape `(2, 4, 768)`

This confirms that the block preserves dimensions - it transforms the representations without changing their shape!

In [11]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


---

## 📝 Summary

### What We Built

We combined all our building blocks into a **TransformerBlock** - the fundamental repeating unit of GPT:

```
┌────────────────────────────────────────────────────────────┐
│                    TransformerBlock                        │
│                                                            │
│   ┌──────────┐   ┌──────────┐   ┌──────────┐   ┌────────┐ │
│   │LayerNorm │ → │   MHA    │ → │ Dropout  │ → │   +    │ │
│   └──────────┘   └──────────┘   └──────────┘   └────────┘ │
│        │                                           ▲       │
│        └───────────── Residual ────────────────────┘       │
│                                                            │
│   ┌──────────┐   ┌──────────┐   ┌──────────┐   ┌────────┐ │
│   │LayerNorm │ → │   FFN    │ → │ Dropout  │ → │   +    │ │
│   └──────────┘   └──────────┘   └──────────┘   └────────┘ │
│        │                                           ▲       │
│        └───────────── Residual ────────────────────┘       │
└────────────────────────────────────────────────────────────┘
```

### Key Takeaways

1. **Pre-LN Architecture**: LayerNorm comes *before* each sub-layer (not after)
   - This is what GPT-2 uses
   - Makes training more stable

2. **Two Sub-Blocks**: Each TransformerBlock has:
   - **Attention sub-block**: Allows tokens to communicate
   - **FeedForward sub-block**: Processes each token independently

3. **Residual Connections**: 
   - Add the input directly to the output
   - Creates "gradient highways" for deep networks
   - Essential for training 12+ layer models

4. **Shape Preservation**: Input and output have the same shape
   - This allows stacking multiple blocks!

### Next Steps

In the next chapter, we'll stack 12 of these TransformerBlocks together with:
- Token embeddings
- Position embeddings  
- Final LayerNorm
- Output projection (language model head)

...to create the complete **GPT-2 model**! 🚀